In [1]:
#to open cred files
import os
#cleaning and operations
import pandas as pd
import numpy as np
#to read translations
import json
import unidecode

In [2]:
#bigquery
import pandas_gbq
from google.cloud import bigquery

In [3]:
#FIREBASE
import firebase_admin

from firebase_admin import credentials
from firebase_admin import firestore

In [4]:
from dotenv import load_dotenv
load_dotenv('/Users/daniel/OAN/credentials/contoan/.env')

True

In [5]:
## OPEN CONECTION TO BIGQUERRY
client = bigquery.Client()
dataset_id = "{}.contoan".format(client.project)
dataset = bigquery.Dataset(dataset_id)

In [7]:
filename=os.environ['FIREBASE_FILENAME']
#OPEN CONECTION TO FIREBASE
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

## Accounting table

### GET DATA

In [74]:
#get all the collection
col_query = db.collection('Accounting').stream()

In [75]:
#save it in a list of dictionaries
accounts = []
for acc in col_query:
    accounts.append(acc.to_dict())

In [76]:
#convert to dataframe
accounting = pd.DataFrame(accounts)

In [73]:
#errores_ids = accounting[(accounting['amount'].apply(lambda x: type(x) == type('2')))]['id'].tolist()
#for error in errores_ids:
#    value = float(accounting[accounting['id']==error]['amount'])
#    db.collection('Accounting'
#                 ).document(error
#                           ).update({'amount':value}
#                                   )

In [78]:
#nulls. size and datatype
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339 entries, 0 to 2338
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   origin          2339 non-null   object 
 1   execution_date  2339 non-null   object 
 2   description     2292 non-null   object 
 3   place           2339 non-null   object 
 4   creator_user    2339 non-null   object 
 5   vat             2116 non-null   float64
 6   id              2339 non-null   object 
 7   amount          2339 non-null   float64
 8   target_id       2339 non-null   object 
 9   images          2336 non-null   object 
 10  type            2339 non-null   object 
 11  phase           768 non-null    object 
 12  concept         2339 non-null   object 
 13  project         2339 non-null   object 
 14  vat_amount      2248 non-null   float64
 15  intervention    2339 non-null   object 
 16  code            2339 non-null   int64  
 17  creation_date   2339 non-null   o

In [13]:
#nulls heatmap - import seaborn as sns
#sns.heatmap(accounting.isna())

### CLEAN DATA

In [79]:
#lets see how it shows
accounting.head()

,origin,execution_date,description,place,creator_user,vat,id,amount,target_id,images,type,phase,concept,project,vat_amount,intervention,code,creation_date,user_in_charge,account_id
0,script_dani,2020-04-28T12:07:49.000Z,venta online: 2288996286563,spain,z5m936GA0t3vHM28QKhR,21.0,01WVDHF0gSSg1PCFsosk,21.0,yfy4cPxkmFwIYVnoWD0A,[],income,None,venta shopify,0DmODGTOEiM5lg9SGx0J,3.644628,w04441aFcU5b7pQm6Rd2,899,2020-08-26T18:02:55.591Z,IjBxXuBshlfq2MUzwBSK,3deqBmrZ1iJkAwMmxSOp
1,shopify,2020-12-22T08:59:35.685Z,2915979624547,spain,z5m936GA0t3vHM28QKhR,0.0,01dtZiGq12IzmIpOVG3X,6.1,iA9Pzv2CImjItzwCaQv0,[],movement,None,Gastos de envio Nacional,0DmODGTOEiM5lg9SGx0J,0.000000,w04441aFcU5b7pQm6Rd2,2257,2020-12-22T08:59:35.685Z,IjBxXuBshlfq2MUzwBSK,FHwFsBItyEneaZh8yP0T
2,shopify,2020-12-07T08:56:28.137Z,2875805401187,spain,z5m936GA0t3vHM28QKhR,0.0,01nQGx25hZLez38FpWEe,6.1,iA9Pzv2CImjItzwCaQv0,[],movement,None,Gastos de envio Nacional,0DmODGTOEiM5lg9SGx0J,0.000000,w04441aFcU5b7pQm6Rd2,1959,2020-12-07T08:56:28.137Z,IjBxXuBshlfq2MUzwBSK,FHwFsBItyEneaZh8yP0T
3,script_dani,2020-04-28T19:50:24.000Z,venta online: 2290422218851,spain,z5m936GA0t3vHM28QKhR,21.0,026RXV3hyl33ussoU7D9,38.0,iA9Pzv2CImjItzwCaQv0,[],income,None,venta shopify,0DmODGTOEiM5lg9SGx0J,6.595041,w04441aFcU5b7pQm6Rd2,1049,2020-08-26T18:02:55.591Z,IjBxXuBshlfq2MUzwBSK,3deqBmrZ1iJkAwMmxSOp
4,shopify,2020-12-05T22:24:00.338Z,2872447467619,spain,z5m936GA0t3vHM28QKhR,0.0,02EE48NlGSbqGrrLvIHl,1.2,iA9Pzv2CImjItzwCaQv0,[],expense,LXspD4R7rZ6Jw5EOEZYR,Comision Stripe,0DmODGTOEiM5lg9SGx0J,0.000000,w04441aFcU5b7pQm6Rd2,1934,2020-12-05T22:24:00.338Z,IjBxXuBshlfq2MUzwBSK,j2UYEKf01eFHMPZv9qHq


#### convert execution to isostring

In [80]:
def clean_execution(x):
    if(len(x['execution_date'])<15):
        doc_id = x['id']
        exec_date=x['execution_date']+'T14:14:58.593Z'
        print(exec_date)
        print(len(x['execution_date']))
        #upload to firebase if whished
        #a=db.collection(u'Accounting').document(doc_id).update({'execution_date':exec_date})
        
        return exec_date
    else:
        return x['execution_date']
        

In [81]:
accounting['execution_date']=accounting.apply(lambda x: clean_execution(x) ,axis=1)

2020-10-31T14:14:58.593Z
10
2020-11-25T14:14:58.593Z
10
2020-09-30T14:14:58.593Z
10
2020-09-01T14:14:58.593Z
10
2020-10-15T14:14:58.593Z
10
2020-10-31T14:14:58.593Z
10


##### clean VAT and replace nan with 0

In [82]:
def clean_vat_amount(x):
    doc_id = x['id']
    amount=x['amount']
    origin = x['origin']
    if origin =='script_dani':
        if x['vat']>0:
            vat=21
        else:
            vat=0
        #a=db.collection(u'Accounting').document(doc_id).update({'vat':vat})
    else:
        vat=x['vat']
        if not(vat >=0):
            vat=0
    vat_amount=amount*(1-1/(1+vat/100))

    #a=db.collection(u'Accounting').document(doc_id).update({'vat_amount':vat_amount})

    return pd.Series({'vat':vat,'vat_amount':vat_amount})

In [83]:
accounting[['vat','vat_amount']] = accounting.apply(lambda x: clean_vat_amount(x),axis=1)

###### convert dates to datetime

In [84]:
#utc=True important if dates are in diferent formats
accounting['creation_date'] = pd.to_datetime(accounting['creation_date'],utc=True)
accounting['execution_date'] = pd.to_datetime(accounting['execution_date'],utc=True)

In [85]:
def getimages(x):
    if not (x):
        return ''
    elif(len(x)==0):
        return ''
    elif('download_url' in x[0].keys()):
        download_url = x[0]['download_url']
        if(download_url == None):
            return ''
        else:
            return download_url
    else:
        return ''

In [86]:
accounting['receipt']=accounting.images.apply(lambda x: getimages(x))

##### double check

In [87]:
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339 entries, 0 to 2338
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   origin          2339 non-null   object             
 1   execution_date  2339 non-null   datetime64[ns, UTC]
 2   description     2292 non-null   object             
 3   place           2339 non-null   object             
 4   creator_user    2339 non-null   object             
 5   vat             2339 non-null   float64            
 6   id              2339 non-null   object             
 7   amount          2339 non-null   float64            
 8   target_id       2339 non-null   object             
 9   images          2336 non-null   object             
 10  type            2339 non-null   object             
 11  phase           768 non-null    object             
 12  concept         2339 non-null   object             
 13  project         2339 non-null   o

### select columns

In [88]:
#and the order whe want to upload to bigquery
df = accounting[['concept', 'place', 'creation_date', 'vat', 'description', 'amount',
       'id', 'phase', 'creator_user', 'execution_date',
       'user_in_charge', 'origin', 'account_id', 'project', 'code', 'type',
       'intervention', 'target_id', 'vat_amount', 'receipt']]

## upload to BIGQUERY

In [89]:
table_id = "{}.accounting".format(dataset.dataset_id)

pandas_gbq.to_gbq(df, table_id, project_id=client.project, if_exists='replace')

1it [00:03,  3.06s/it]


### SUBIR VENTAS

#### get data

In [8]:
col_query = db.collection('Nikarit_Sales').stream()

In [9]:
ventas = []
for acc in col_query:
    ventas.append(acc.to_dict())

In [10]:
df_ventas = pd.DataFrame(ventas)

In [11]:
df_ventas.shape

(852, 57)

### view

In [12]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 57 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   manteca_nb               852 non-null    int64  
 1   gateway                  852 non-null    object 
 2   buyer_accepts_marketing  216 non-null    object 
 3   latitude                 748 non-null    float64
 4   packHidra_nb             852 non-null    int64  
 5   id                       852 non-null    object 
 6   subtotal_price           852 non-null    object 
 7   order_status_url         817 non-null    object 
 8   balsamo_unit_price       852 non-null    object 
 9   crema_unit_price         852 non-null    object 
 10  product_movement         205 non-null    object 
 11  packContigo_unit_price   852 non-null    object 
 12  creation_date            852 non-null    object 
 13  accounting_item          217 non-null    object 
 14  order_id                 8

#### transform data

In [13]:
cols = list(df_ventas.columns)

In [14]:
cambios = [col for col in cols if 'price' in col]

In [15]:
for cambio in cambios:
    df_ventas[cambio]=pd.to_numeric(df_ventas[cambio])

In [16]:
df_ventas['creation_date']=pd.to_datetime(df_ventas['creation_date'],utc=True)
df_ventas['close_date']=pd.to_datetime(df_ventas['close_date'],utc=True)

In [17]:
#double check
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 57 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   manteca_nb               852 non-null    int64              
 1   gateway                  852 non-null    object             
 2   buyer_accepts_marketing  216 non-null    object             
 3   latitude                 748 non-null    float64            
 4   packHidra_nb             852 non-null    int64              
 5   id                       852 non-null    object             
 6   subtotal_price           852 non-null    float64            
 7   order_status_url         817 non-null    object             
 8   balsamo_unit_price       852 non-null    float64            
 9   crema_unit_price         852 non-null    float64            
 10  product_movement         205 non-null    object             
 11  packContigo_unit_price   852 non

In [18]:
df_ventas.shape

(852, 57)

### METER MUJERES Y HOMBRES

In [19]:
#client_gender

In [20]:
#lista de mujeres
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/mujeres.csv'
df_mujeres = pd.read_csv(url, error_bad_lines=False)
l_mujeres = df_mujeres['nombre'].to_list()

In [21]:
l_mujeres = [str(nom).lower() for nom in l_mujeres]

In [22]:
l_mujeres= l_mujeres+['maasuncion','mirenchu','begona','nadai','sra.','mcarmen','carmena','maita',
'madolores','reyero','ma','maconcepción','cris']

In [23]:
#lista de hombres
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/hombres.csv'
df_hombres = pd.read_csv(url, error_bad_lines=False)
l_hombres = df_hombres['nombre'].to_list()

In [24]:
l_hombres = [nom.lower() for nom in l_hombres]

In [25]:
l_hombres= l_hombres+['fcojavier']

In [26]:
#aplicar sexo
df_ventas['client_gender'] = df_ventas['client_name'].apply(lambda x: 
                            'F' if unidecode.unidecode(
                                    str(x).replace(" ", "").lower()) in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x).replace(" ", "").lower())  in l_hombres
                            else 'F' if unidecode.unidecode(
                                str(x).split(" ")[0].lower())  in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x).split(" ")[0].lower())  in l_hombres

                            else 'N/A')

In [27]:
#ver que sale
df_ventas['client_gender'].value_counts() # A MEJORRAR

F      663
M      166
N/A     23
Name: client_gender, dtype: int64

In [28]:
df_ventas[df_ventas['client_gender'] == 'N/A'
         ]['client_name'].apply(lambda x: unidecode.unidecode(str(x).replace(" ", "").lower()))

55                  chinwe
65              herguedas,
66        notariaserrano19
76       impulsandolab,s.l
104         beaaccemmurcia
311         farmacialaluna
316                herrera
321                 gandou
331                   none
389                  durga
408                   none
412                   none
419                   none
428       laposadademanolo
455                   none
474            ventasorteo
483              ventajose
507                   none
574                 clavis
585    herbolariosalamanca
757               vinuales
803                   none
846                   saez
Name: client_name, dtype: object

## METER DONACIONES

In [29]:
df_ventas['origin'].unique()

array(['Shopify', 'Contoan'], dtype=object)

In [30]:
def don(x):
    total_price = x['total_price']
    subtotal_price = x['subtotal_price']
    fecha = str(x['creation_date']).split(' ')[0]
    origin = x['origin']
    don = 0.0
    if '2020-11-01' < fecha and origin == "Shopify":
        if total_price < 40:
            don = total_price - subtotal_price - 5.50
        else:
            don = total_price - subtotal_price
            
    return don

df_ventas['donation'] = df_ventas.apply(lambda x: don(x),axis=1)

In [31]:
df_ventas['donation'] = df_ventas.apply(lambda x: don(x),axis=1)

In [32]:
df_ventas['donation'].sum()

227.42999999999998

## ERROR EN PROVINCE

In [33]:
import requests

In [34]:
shop_url = os.environ['SHOPIFY_ACCESS_URL']
dates = ['2019-01-01T00:15:47-04:00','2020-01-01T00:15:47-04:00',
         '2020-05-01T00:15:47-04:00','2020-12-10T00:15:47-04:00', '2021-01-10T00:15:47-04:00']
l_orders = []

for date_min,date_max in zip(dates[:-1],dates[1:]):
    orders_url = shop_url+'/orders.json'
    r = requests.get(orders_url,
                 params={
                        'limit':250,
                        'status':'any',
                         'created_at_max':date_max,
                         'created_at_min':date_min
                        })
    orders = pd.DataFrame(r.json()['orders'])
    l_orders.append(orders)
df_orders = pd.concat(l_orders,axis=0).reset_index().drop_duplicates(subset=['id'])

In [35]:
df_orders['province_2'] = df_orders['billing_address'].apply(lambda x: x['province'])
df_orders['order_id'] = df_orders['id'].apply(str)
df_ventas['order_id'] = df_ventas['order_id'].apply(str)
df_ventas = pd.merge(df_ventas,df_orders[['order_id','province_2']],on='order_id',how='left')

df_ventas['province'] = df_ventas['province_2'] 
df_ventas = df_ventas.drop('province_2',axis=1)

In [36]:
cols = list(df_ventas.columns)
cols.pop(cols.index('to'))
cols.pop(cols.index('message'))
cols.pop(cols.index('navidadCode'))

'navidadCode'

In [37]:
df_ventas = df_ventas[cols]

In [38]:
df_ventas.shape

(852, 54)

In [39]:
df_ventas['province'].value_counts().head()

Madrid        427
Barcelona      45
Valladolid     42
Valencia       30
Málaga         22
Name: province, dtype: int64

## ACTUALIZAR FIREBAE

In [40]:
#l_ship_det = ['ship_'+a for a in list(df_ord['shipping_address'][0].keys())]
#df_ord['ship_zip']=df_ord['ship_zip'].fillna(value='')
#def update_firebase(x):
#    order_id = x['order_id']
#    if order_id==None :
#        return True
#    else:
#        order_id = int(order_id)
#    ship_zip=df_ord[df_ord['id']==order_id]['ship_zip'].values[0]
#    fire_id=x['id']
#    #print(x['creation_date'])
#    #a=db.collection(u'Nikarit_Sales').document(x['id']).update({'zip':ship_zip})
##df_ventas[df_ventas['origin']!='Contoan'].apply(lambda x: update_firebase(x),axis=1)

## CORREGIR SUBTOTAL Y IVA

In [41]:
def iva(x):
    iva = x['total_tax']
    
    #Corregir ventas de contoan
    if x['origin'] == 'Contoan':
        iva = x['total_price']*(1-1/1.21)
    return iva

df_ventas["total_tax_2"] = df_ventas.apply(lambda x: iva(x),axis=1)
df_ventas["total_tax"] = df_ventas["total_tax_2"]
df_ventas = df_ventas.drop('total_tax_2',axis=1)

In [42]:
def subtotal(x):
    subtotal = x['subtotal_price']
    
    #Corregir ventas de contoan
    if x['origin'] == 'Contoan':
        subtotal = x['total_price']
    return subtotal

df_ventas["subtotal_price_2"] = df_ventas.apply(lambda x: subtotal(x),axis=1)
df_ventas["subtotal_price"] = df_ventas["subtotal_price_2"]
df_ventas = df_ventas.drop('subtotal_price_2',axis=1)

In [43]:
#subir a bigquery
table_id = "contoan.sales"

pandas_gbq.to_gbq(df_ventas, table_id, project_id=client.project, if_exists='replace')

1it [00:07,  7.33s/it]


## TRANSFORM BOOLEAN

In [263]:
sql = """
CREATE OR REPLACE TABLE `oan-nikarit.contoan.sales` AS
SELECT
  * EXCEPT (buyer_accepts_marketing,taxes_included),
  CAST(buyer_accepts_marketing AS BOOL) AS buyer_accepts_marketing,
  CAST(taxes_included AS BOOL) AS taxes_included
FROM
  `oan-nikarit.contoan.sales`
"""

In [264]:
client.query(sql)

## Cuentas

In [7]:
#get all the collection
col_query = db.collection('Accounts').stream()

In [8]:
#save it in a list of dictionaries
cuentas = []
for acc in col_query:
    cuentas.append(acc.to_dict())

In [9]:
#convert to dataframe
df_accounts = pd.DataFrame(cuentas)

In [34]:
df_cuentas = df_accounts[['id','name','number','type','user_id']].copy()

In [35]:
with open('fr.json', 'r') as f:
    trads = json.load(f)

In [36]:
df_cuentas['name_fr'] = df_cuentas['name'].apply(lambda x: trads[x])

In [37]:
df_cuentas[df_cuentas['name_fr'].isnull()]

,id,name,number,type,user_id,name_fr


In [40]:
table_id = "{}.accounts".format(dataset.dataset_id)

pandas_gbq.to_gbq(df_cuentas, table_id, project_id=client.project, if_exists='replace')

1it [00:05,  5.93s/it]


### PROYECTOS

In [41]:
#get all the collection
col_query = db.collection('Projects').stream()

In [42]:
#save it in a list of dictionaries
projects = []
for acc in col_query:
    projects.append(acc.to_dict())

In [43]:
#convert to dataframe
df_projects = pd.DataFrame(projects)

In [44]:
df_projectos=df_projects[['id','title','description','department','type','project_id','intervention_id']]

In [45]:
df_projectos.columns = ['id', 'name', 'description', 'department', 'type', 'project_id',
       'intervention_id']

In [46]:
table_id = "{}.projects".format(dataset.dataset_id)

pandas_gbq.to_gbq(df_projectos, table_id, project_id=client.project, if_exists='replace')

1it [00:03,  3.59s/it]


### USUARIOS

In [50]:
#get all the collection
col_query = db.collection('Users').stream()

In [51]:
#save it in a list of dictionaries
usuarios = []
for acc in col_query:
    usuarios.append(acc.to_dict())

In [52]:
#convert to dataframe
df_usuarios = pd.DataFrame(usuarios)

In [53]:
df_users = df_usuarios[['id','name','last_name','email']]

In [54]:
df_users_final = pd.merge(df_users,df_cuentas[['number','user_id']],left_on='id',right_on='user_id',how='left')

In [55]:
df_users = df_users_final[['id','name','last_name','email','number']]

In [56]:
df_users.columns = ['id','name','last_name','email','cuenta']

In [58]:
table_id = "{}.users".format(dataset.dataset_id)

pandas_gbq.to_gbq(df_users, table_id, project_id=client.project, if_exists='replace')

1it [00:06,  6.33s/it]


##### september 2020 backup

In [67]:
table_id = "{}.accounting_202009".format(dataset.dataset_id)

pandas_gbq.to_gbq(df, table_id, project_id=client.project, if_exists='replace')

1it [00:04,  4.48s/it]


##### Jan 2021 backup

In [351]:
sql = """
CREATE TABLE AS `oan-nikarit.contoan.accounting_20210110`
AS
SELECT *
FROM `oan-nikarit.contoan.accounting`
"""

In [352]:
client.query(sql)